In [1]:
import matplotlib.pyplot as plt
import numpy as np
import skimpy

In [2]:
x = skimpy.IntArray(10, 1)

In [3]:
x

IndexError: invalid vector<T> subscript

In [5]:
%%time
x[3:6] = 2

Wall time: 0 ns


In [6]:
x

Array<int>([1, 1, 1, 2, 2, 2, 1, 1, 1, 1])

In [7]:
%%time
x[5:9] = 3

Wall time: 0 ns


In [8]:
%%time
x[1:9:2]

Wall time: 0 ns


Array<int>([1, 2, 3, 3])

In [15]:
%%time
x[1:9:2][1::2]

Wall time: 0 ns


Array<int>([2, 3])

In [16]:
%%time
x[-6:-2:3]

Wall time: 0 ns


Array<int>([2, 3])

In [17]:
%%time
2 * x

Wall time: 0 ns


Array<int>([2, 2, 2, 4, 4, 6, 6, 6, 6, 2])

In [18]:
%%time
y = 2 * x

Wall time: 0 ns


In [19]:
y

Array<int>([2, 2, 2, 4, 4, 6, 6, 6, 6, 2])

In [20]:
x

Array<int>([1, 1, 1, 2, 2, 3, 3, 3, 3, 1])

In [21]:
%%time
((x + y)**x + 56 * x - 3 * x) // x**2

TypeError: unsupported operand type(s) for ** or pow(): 'skimpy.IntArray' and 'skimpy.IntArray'

In [22]:
%%time
builder = skimpy.IntArrayBuilder(1024 * 1024, 1)
for i in range(1024 * 1024):
    builder[i] = i
big = builder.build()

Wall time: 668 ms


In [23]:
%%time
big_np = np.zeros(1024 * 1024, dtype=np.int32)
for i in range(1024 * 1024):
    big_np[i] = i

Wall time: 112 ms


In [24]:
%%time
big = skimpy.from_numpy(big_np)

Wall time: 7 ms


In [25]:
%%time
big = skimpy.from_numpy(np.arange(1024 * 1024))

Wall time: 7.02 ms


In [26]:
%%timeit
(big * big * big * big * big * big * big * big).eval()

42.3 ms ± 556 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%%timeit
(big * big * big * big * big * big * big * big)[100:200].eval()

43.8 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
%%timeit
big_np * big_np * big_np * big_np * big_np * big_np * big_np * big_np

10.5 ms ± 94.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%%time
D = 4096
R = D // 4
builder = skimpy.IntArrayBuilder(D * D, 0)
for y in range(D):
    discriminant = R**2 - (y - D // 2 + 0.5)**2
    if discriminant < 0:
        continue
    x_0 = int(D // 2 - 0.5 - discriminant**0.5)
    x_1 = int(D // 2 - 0.5 + discriminant**0.5)  
    i = D * y + x_0
    j = D * y + x_1
    builder[i:j] = 1
disc = builder.build()

Wall time: 13 ms


In [30]:
%%timeit
(disc * disc).eval()

113 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [31]:
%%time
disc_np = disc.to_numpy()

IndexError: invalid vector<T> subscript

In [32]:
%%timeit
disc_np * disc_np

NameError: name 'disc_np' is not defined

In [ ]:
%%time
img = disc.to_numpy().reshape(D, D)

In [ ]:
plt.imshow(img)

In [ ]:
shift_x = disc.clone()
shift_x[D // 10:] = disc[:-(D // 10)]
plt.imshow(shift_x.to_numpy().reshape(D, D))

In [ ]:
shift_y = disc.clone()
shift_y[D // 8 * D:] = disc[:-D // 8 * D]
plt.imshow(shift_y.to_numpy().reshape(D, D))

In [ ]:
plt.imshow((disc | shift_x).to_numpy().reshape(D, D))

In [ ]:
plt.imshow(((disc | shift_x) & ~shift_y).to_numpy().reshape(D, D))

In [ ]:
%%timeit
len(((disc | shift_x) & ~shift_y).to_numpy().dumps())

In [ ]:
%%timeit
len(((disc | shift_x) & ~shift_y).dumps())

In [ ]:
%%timeit
((disc | shift_x) & ~shift_y).eval()